* 색인 & 가중치 => 학습
1. 수집한 문서 목록 가져오기
2. 각 문서에서 색인어 목록 추출 전처리 -> 어절 -> 형태소+명사+바이그램
3. 가중치 계산(TF-IDF) -> TDM -> TWM(with DocumentVectorLength)

* 질의 -> 색인과정의 2 to 3
4. 질의에서 색인어 추출
5. 가중치 계산
6. 유사도 계산(코싸인)
7. 유사도 순 정렬(거리-오름차순, 각도-내림차순)
8. 검색 결과 출력

In [3]:
def getFileList(base = "./", ext = "txt"): # 아무것도 안했다면 base는 현재 경로
    fileList = list()
    for file in listdir(base):
        if file.split(".")[-1] == ext: # .을 기준으로 split한 것이 txt인지 검사
            fileList.append("{0}/{1}".format(base, file))
            
    return fileList

# txt 컨텐츠를 편하게 읽어오기 위한 함수
def getContent(file): 
    with open(file, encoding="UTF-8") as f:
        content = f.read()
    return content

In [4]:
def ngramEojeol(sentence, n=2): # sentence를 받아 어절 단위로 분리해주는 함수
    '''
    입력:     단어1,   단어2,   단어3,  단어4 : 4
    출력(2) : 단어12,  단어23,  단어34 :        3 - n + 1
    출력(3) : 단어123, 단어234         :        2 - n + 1
    '''
    tokens = sentence.split()
    ngram = []
    
    for i in range(len(tokens) - n + 1):
        ngram.append(' '.join(tokens[i:i + n]))    
        
    return ngram

In [5]:
def ngramUmjeol(term, n = 2): # 음절 단위로 구분하는 함수. sentence를 받아 2개(n=2)씩 쪼갠다.

    ngram = []
    
    for i in range(len(term) - n + 1):
        # ngram.append(tokens_ngram[i:i+n]) # 방법1
        # ngram.append(tuple(tokens_ngram[i:i+n])) # 방법2 (튜플로 반환 시 키값을 쓸 수 있음)
        ngram.append(''.join(term[i:i + n])) # 방법3
        
    return ngram

In [6]:
from os import listdir

FileList = getFileList("C:/Users/brsta/ICT_AI_AdvanceClass_NLP/0314_DownloadedNewstxts/")
NewsContent = list() 
for i in range(len(FileList)):
    NewsContent.append(getContent(FileList[i])) # len(NewsContent) = 180

In [7]:
import re
from string import punctuation


def getPatternList():
    patternList = dict()
    pattern = re.compile(r"[%s]{1,}" % re.escape(punctuation)) # punctuation 안의 특수문자가 두번이상 반복되는 모든 문자에 대해 패턴 정의
    patternList["Punctuation"] = pattern
    pattern2 = re.compile(r"([A-Za-z0-9\-\_\.]{3,}@[A-Za-z0-9\-\_]{3,}(.[A-Za-z]{2,})+)") #이메일주소제거패턴
    patternList["Email"] = pattern2
    pattern3 = re.compile(r"([A-Za-z0-9\-\_]{1,}(.[A-Za-z]{2,})+)") # 신문사도메인패턴
    patternList["Domain"] = pattern3
    pattern4 = re.compile(r"\s{2,}") # 공백제거
    patternList["Whitespace"] = pattern4
    pattern5 = re.compile(r"([^ㄱ-ㅎㅏ-ㅣ가-힣0-9]+)") # 한글이 아닌 영어 기호 제거
    patternList["Korean"] = pattern5
    pattern6 = re.compile(r'[A-Za-z-_]{8,}') # 영어 (대소문자) +\-\_ 제거 (8글자 이상)
    patternList["ElimLongEng"] = pattern6
    pattern7 = re.compile(r"를 2019 및 금지|네이버 채널에서|저작권자 주 블로터앤미디어 저작권자를 명기하고 내용을 변경하지 않으며 비상업적으로 이용하는 조건아래 가능합니다|"
                          r"디지털 마케팅의 미래 아이포럼|무단전재|오류를 우회하기 위한 함수 추가|재배포|기자|아이뉴스24|구독해주세요|"
                          r"고수들의 재테크 비법|이건희칼럼|성공을 꿈꾸는 사람들의 경제 뉴스|머니 및 금지|"
                          r"네이버 모바일에서|네이버에서 헤럴드경제|채널 구독하기|헤럴드 리얼라이프 헤럴드경제 사이트 바로가기 헤럴드경제 무단 전재 및 금지|"
                          r"전자신문|바로가기|전자신문인터넷|인터넷 및 금지|중앙일보|친구추가|구독 1위|") # 높은 가중치를 갖는 단어를 하드코딩 데이터로 제거
    patternList["ElimRecWord"] = pattern7
    
    return patternList

In [8]:
patternList = getPatternList()
Counter = 0
for _ in ["Korean", "Whitespace", "Punctuation", "ElimLongEng", "Email", "Domain", "ElimRecWord"]:
    for i in range(len(FileList)):
        NewsContent[i] = patternList[_].sub(" ", NewsContent[i]) 
print(NewsContent[11])

      애 플   전 세 계   과 학 자 들 이   애 플   에 어 팟   등   무 선   이 어 폰 이   암   발 생   위 험 을   키 울   수   있 다 는   호 소 문 을   유 럽 연 합   과   세 계 보 건 기 구   에   제 출 했 다   전 세 계   4 0 여 개 국   생 물   건 강   분 야   과 학 자   2 5 0 여 명 은   최 근   무 선 장 치 에 서   발 생 하 는   비 이 온 화   전 자 기 장   가   암   유 발   가 능 성 이   있 다   며   심 각 한   우 려   를   표 명 했 다   는   전 기 장 치   등 에 서   발 생 하 는   극 저 주 파   전 자 기 장   와   블 루 투 스   와 이 파 이   안 테 나   기 지 국   등 에 서   나 오 는   고 주 파   방 사 선   을   말 한 다   과 학 자 들 은   애 플   에 어 팟   등   무 선   이 어 폰 이   두 뇌 에   어 떤   영 향 을   미 치 는 지   아 직   연 구 결 과 가   없 다   면 서 도   을   동 물 들 에 게   노 출 한   결 과   생 식 적   신 경 적   유 전 적   손 상 이   나 타 난   것 을   발 견 했 다   고   설 명 했 다   에 어 팟 은   에   관 한   법 적   기 준 치 를   준 수 하 고   있 다   하 지 만   과 학 자 들 은   가   기 준 치 보 다   낮 아 도   암 을   유 발 할   가 능 성 은   있 으 며   낮 은   수 준 의   노 출 이   장 시 간   이 뤄 질   경 우   건 강 에   좋 지   않 은   영 향 을   미 칠   수 도   있 음 을   고 려 해 야   한 다 고   주 장 한 다   특 히   에 어 팟 은   귓 구 멍   안 에   깊 게   들 어 가   두 뇌 가   노 출   위 험 에   더 욱   취 약 하 다 